In [1]:
import nest_asyncio
nest_asyncio.apply()
from X_application_directory.src.snag_tokens import Usage

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

import logging
import sys
from IPython.display import Markdown

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from llama_index import ListIndex
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from llama_index.schema import MetadataMode
from llama_index.response import pprint_utils as pp

In [5]:
from llama_index.llms import OpenAI

llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=512)

In [6]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    KeywordExtractor,
    MetadataFeatureExtractor,
)
from llama_index.langchain_helpers.text_splitter import SentenceSplitter


text_splitter = SentenceSplitter(
  chunk_size=512,
  chunk_overlap=5,
)

In [7]:
metadata_extractor = MetadataExtractor(
    extractors=[
        SummaryExtractor(summaries=["self"]),
        KeywordExtractor(keywords=5),
    ],
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

In [8]:
from llama_index import SimpleDirectoryReader, DocumentSummaryIndex

In [9]:
from pathlib import Path

documents_ho3 = SimpleDirectoryReader(input_files=['./data/HO3_sample.pdf']).load_data()

In [10]:
nodes_ho3 = node_parser.get_nodes_from_documents(documents_ho3)

In [11]:
nodes_ho3[0].metadata

{'page_label': '1',
 'file_name': 'HO3_sample.pdf',
 'section_summary': '\nThis section defines the terms and conditions of the Homeowners 3 Special Form Agreement. It outlines the insurance coverage provided in exchange for the premium and compliance with the policy. It defines the terms "you" and "your" as the named insured and their spouse, and "we", "us" and "our" as the company providing the insurance. It also defines the terms "Aircraft Liability", "Hovercraft Liability", "Motor Vehicle Liability" and "Watercraft Liability" as liability for bodily injury or property damage arising from the ownership, maintenance, occupancy, operation, use, loading or unloading of such vehicles or crafts. It also defines the terms "Bodily Injury" as bodily harm, sickness or disease, including required care, loss of services and death that results.',
 'excerpt_keywords': ' Homeowners, Insurance, Liability, Aircraft, Watercraft'}

In [12]:
nodes_ho3[1].metadata

{'page_label': '1',
 'file_name': 'HO3_sample.pdf',
 'section_summary': '\nThis section defines the terms "business," "employee," and "insured." A business is any activity engaged in for money or other compensation, except for certain volunteer activities, providing home day care services, or rendering home day care services to a relative. An employee is an employee of an insured or an employee leased to an insured by a labor leasing firm. An insured is the policyholder and their relatives, as well as a student enrolled in school full time who was a resident of the policyholder\'s household before moving out to attend school.',
 'excerpt_keywords': '\nBusiness, Employee, Insured, Relative, Compensation'}

In [13]:
print(nodes_ho3[0].text)

HOMEOWNERS
HO 00 03 10 00
HO 00 03 10 00 Copyright, Insurance Services Office, Inc. 1999 Page 1 of 22HOMEOWNERS 3 – SPECIAL FORM
AGREEMENT
We will provide the insurance described in this policy
in return for the premium and compliance with allapplicable provisions of this policy.
DEFINITIONS
A.In this policy, "you" and "your" refer to the "named
insured" shown in the Declarations and the spouseif a resident of the same household. "We", "us"and "our" refer to the Company providing this in-surance.
B.In addition, certain words and phrases are definedas follows:
1."Aircraft Liability", "Hovercraft Liability", "Motor
Vehicle Liability" and "Watercraft Liability",subject to the provisions in b. below, mean the
following:
a.Liability for "bodily injury" or "property dam-age" arising out of the:
(1)Ownership of such vehicle or craft by an"insured";
(2)Maintenance, occupancy, operation,use, loading or unloading of such vehi-cle or craft by any person;
(3)Entrustment of such vehicle or craft by

In [14]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import FaissVectorStore
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata

In [15]:
index = VectorStoreIndex(nodes=nodes_ho3)
engine = index.as_query_engine(similarity_top_k=10)

In [16]:
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.prompts import DEFAULT_SUB_QUESTION_PROMPT_TMPL

service_context = ServiceContext.from_defaults(
    llm=llm, node_parser=node_parser
)
question_gen = LLMQuestionGenerator.from_defaults(
    service_context=service_context,
    prompt_template_str="""
        Follow the example, but instead of giving a question, always prefix the question 
        with: 'By first identifying and quoting the most relevant sources, '. 
        """
    + DEFAULT_SUB_QUESTION_PROMPT_TMPL,
)

In [17]:
DEFAULT_SUB_QUESTION_PROMPT_TMPL

'Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\n\n# Example 1\n<Tools>\n```json\n{{\n    "uber_10k": "Provides information about Uber financials for year 2021",\n    "lyft_10k": "Provides information about Lyft financials for year 2021"\n}}\n```\n\n<User Question>\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\n\n\n<Output>\n```json\n[\n    {{\n        "sub_question": "What is the revenue growth of Uber",\n        "tool_name": "uber_10k"\n    }},\n    {{\n        "sub_question": "What is the EBITDA of Uber",\n        "tool_name": "uber_10k"\n    }},\n    {{\n        "sub_question": "What is the revenue growth of Lyft",\n        "tool_name": "lyft_10k"\n    }},\n    {{\n        "sub_question": "What is the EBITDA of Lyft",\n        "tool_name": "lyft_10k"\n    }}\n]\n```\n\n# Example 2\n<Tools>\n```json\n{tools_str}\n```\n\n<User Question>\n{query_str}\n\n

In [18]:
final_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name="homeowners_policy_documents",
                description="insurance policy contract detailing coverage provisions.",
            ),
        )
    ],
    question_gen=question_gen,
    use_async=True,
)

In [19]:
response = final_engine.query(
    """
    How does the policy addresses water damage for dwelling damage and personal property damage?
    Include a bullet point list of citations in your answer.
    """
)
print(response.response)

Generated 2 sub questions.
[homeowners_policy_documents] Q: By first identifying and quoting the most relevant sources, what does the homeowners policy documents say about water damage for dwelling damage?
[homeowners_policy_documents] Q: By first identifying and quoting the most relevant sources, what does the homeowners policy documents say about water damage for personal property damage?
[homeowners_policy_documents] A: 

The homeowners policy documents state that water damage to personal property is covered under Coverage C – Personal Property, with the exception of loss to the system or appliance from which the water or steam escaped. This is outlined in Section I – Property Coverages, which states: "(5)Mold, fungus or wet rot. However, we do insure for loss caused by mold, fungus or wet rot that is hidden within the walls or ceilings or beneath the floors or above the ceilings of a structure if such loss results from the accidental discharge or overflow of water or steam from wit

In [20]:
pp.pprint_response(response, show_source=True)

Final Response: The homeowners policy documents address water damage
for dwelling damage and personal property damage as follows:
Dwelling Damage:  • Coverage A – Dwelling, Section I – Property
Coverages outlines that the dwelling on the "residence premises" is
covered, including structures attached to the dwelling, and materials
and supplies located on or next to the "residence premises" used to
construct, alter or repair the dwelling or other structures on the
"residence premises".  • Losses caused by water damage are settled at
actual cash value at the time of loss but not more than the amount
required to repair or replace.  • Losses caused by water damage are
not covered if they are caused by wear and tear, mechanical breakdown,
smog, rust or other corrosion, smoke from agricultural smudging or
industrial operations, discharge, dispersal, seepage, migration,
release or escape of pollutants, settling, shrinking, bulging or
expansion, birds, vermin, rodents, or insects, or animals ow

In [23]:
print(response.response)

The homeowners policy documents address water damage for dwelling damage and personal property damage as follows: 

Dwelling Damage: 
• Coverage A – Dwelling covers direct physical loss to the property caused by perils such as fire or lightning, windstorm or hail, explosion, riot or civil commotion, aircraft, vehicles, and smoke (HO 00 03 10 00, Page 9 of 22). 
• It also covers loss to the dwelling from an accidental discharge or overflow of water or steam from within a plumbing, heating, air conditioning or automatic fire protective sprinkler system, or a household appliance, on the "residence premises" (HO 00 03 10 00, Page 9 of 22). 
• Loss to an awning, fence, patio, deck, pavement, swimming pool, underground pipe, flue, drain, cesspool, septic tank, foundation, retaining wall, bulkhead, pier, wharf or dock is not included unless it is a direct result of the collapse of a building or any part of a building (HO 00 03 10 00, Page 9 of 22). 
• This coverage also covers the breakage of

In [24]:
response = final_engine.query(
    """
    How does the policy addresses water damage for dwelling damage and personal property damage?
    """
)
print(response.response)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": "\\n        Follow the example, but instead of giving a question, always prefix the question \\n        with: \'By first identifying and quoting the most relevant sources, \'. \\n        Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\"

In [25]:
print(response.response)

The homeowners policy document addresses water damage for dwelling damage by covering it under Coverage A – Dwelling, with the exception of land, including land on which the dwelling is located. It also states that water damage resulting from an accidental discharge or overflow of water or steam from within a storm drain, water, steam, or sewer pipe off the residence premises is covered. Additionally, it states that any ensuing loss to property described in Coverages A and B not precluded by any other provision in the policy is covered, with the exception of losses caused by Landslide, mudslide or mudflow, subsidence or sinkhole, or any other earth movement including earth sinking, rising or shifting; water damage caused by or resulting from human or animal forces or any act of nature, including flood, surface water, waves, tidal water, overflow of a body of water, or spray from any of these, whether or not driven by wind; water or water-borne material which backs up through sewers or 

In [26]:
response = final_engine.query(
    """
    What are the differences in how the policy addresses water damage for dwelling damage and personal property damage?
    Include a list of sources, i.e. TITLE, sub-title, A., 1., a, (1)., in your final answer.
    """
)
print(response.response)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": "\\n        Follow the example, but instead of giving a question, always prefix the question \\n        with: \'By first identifying and quoting the most relevant sources, \'. \\n        Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\"

In [27]:
Markdown(f"{response.response}")

The policy addresses water damage for dwelling damage and personal property damage differently. For dwelling damage, the policy covers the cost to tear out and replace any part of a building, or other structure, on the "residence premises", but only when necessary to repair the system or appliance (Section I – Property Coverages, Coverage A). It also covers losses caused by water damage, such as freezing, theft, vandalism, and mold, fungus, or wet rot (Section I – Property Coverages, Coverage A), as well as an accidental discharge or overflow of water or steam from a storm drain or water, steam, or sewer pipe off the "residence premises" (Section I – Property Coverages, Coverage A). Additionally, the policy will pay up to $5,000 for grave markers, including mausoleums, on or away from the "residence premises" for loss caused by a Peril Insured Against under Coverage C (Section I – Property Coverages, Coverage C). The policy also excludes losses caused by water damage, such as flooding, surface water, waves, tidal water, overflow of a body of water, or spray from any of these, whether or not driven by wind, water or water-borne material which backs up through sewers or drains or which overflows or is discharged from a sump, sump pump or related equipment, or water or water-borne material below the surface of the ground, including water which exerts pressure on or seeps or leaks through a building, sidewalk, driveway, foundation, swimming pool or other structure (Section I – Property Coverages, Exclusions). Additionally, the policy excludes losses caused by war, communicable disease, or controlled substances (Section I – Property Coverages, Exclusions). Finally, the policy will pay up to $2,500 for appliances, carpeting, and other household furnishings in each apartment on the "residence premises" regularly rented or held for rental to others, and up to 10% of the limit of liability that applies to Coverage A can be used for increased costs due to the enforcement of any ordinance or law which requires or regulates the construction, demolition, remodeling, renovation, or repair of a covered building or other structure, provided the dwelling is used only as a residence, or in part for use only as a residence, or in part, as an office, school, studio or private garage (Section I – Property Coverages, Coverage A).

For personal property damage, the policy covers direct physical loss to the property caused by any of the perils listed in Coverage C – Personal Property, such as fire or lightning, windstorm or hail, explosion, riot or civil commotion, aircraft, vehicles, and smoke (Section I – Property Coverages, Coverage C). It does not cover loss to the system or appliance from which the water or steam escaped (Section I – Property Coverages, Coverage C). However, it does cover loss caused by mold, fungus, or wet rot that is hidden within the walls or ceilings or beneath the floors or above the ceilings of a structure if such loss results from the accidental discharge or overflow of water or steam from within a plumbing, heating, air conditioning, or automatic fire protective sprinkler system, or a household appliance, on the "residence premises", or a storm drain, or water, steam, or sewer pipes, off the "residence premises" (Section I – Property Coverages, Coverage C). Additionally, it does not cover loss caused by water damage, power failure, landslide, mudslide or mudflow, subsidence or sinkhole, or any other earth movement including earth sinking, rising or shifting, unless direct loss by fire or explosion ensues (Section I – Property Coverages, Coverage C). Furthermore, it excludes loss caused by flood, surface water, waves, tidal water, overflow of a body of water, or spray from any of these, whether or not driven by wind; water or water-borne material which backs up through sewers or drains or which over-flows or is discharged from a sump, sump pump or related equipment; or water or water-borne material below the surface of the ground, including water which exerts pressure on or seeps or leaks through a building, sidewalk, driveway, foundation, swimming pool or other structure; caused by or resulting from human or animal forces or any act of nature (Section I – Property Coverages, Exclusions). Additionally, it excludes power failure if the failure takes place off the "residence premises" and neglect of an "insured" to use all reasonable means to save and preserve property at and after the time of a loss (Section I – Property Coverages, Exclusions). Lastly, it excludes war, including undeclared war, civil war, insurrection, rebellion or revolution, warlike act by a military force or military personnel, or destruction, seizure or use for a military purpose, including the discharge of a nuclear weapon (Section I – Property Coverages, Exclusions). 

In summary, the policy does not cover water damage for personal property damage, except for direct physical loss caused by fire or explosion, or mold, fungus, or wet rot that is hidden within the walls or ceilings or beneath the floors or above the ceilings of a structure (Section I – Property Coverages, Coverage C). 

Sources: 
Section I – Property Coverages, Coverage A
Section I – Property Coverages, Coverage C
Section I – Property Coverages, Exclusions

In [34]:
Markdown(f"{response.source_nodes[0].node.text}")


Sub question: By first identifying and quoting the most relevant sources, what does the policy say about water damage for dwelling damage?
Response: 

The policy states that water damage to the dwelling on the "residence premises" is covered under Coverage A of Section I – Property Coverages. This includes the cost to tear out and replace any part of a building, or other structure, on the "residence premises", but only when necessary to repair the system or appliance. The policy also states that losses caused by water damage, such as freezing, theft, vandalism, and mold, fungus, or wet rot, are covered under Coverage A, unless the loss is otherwise excluded. Additionally, the policy states that an accidental discharge or overflow of water or steam from a storm drain or water, steam, or sewer pipe off the "residence premises" is covered under Coverage A. Furthermore, the policy will pay up to $5,000 for grave markers, including mausoleums, on or away from the "residence premises" for loss caused by a Peril Insured Against under Coverage C. The policy also excludes losses caused by water damage, such as flooding, surface water, waves, tidal water, overflow of a body of water, or spray from any of these, whether or not driven by wind, water or water-borne material which backs up through sewers or drains or which overflows or is discharged from a sump, sump pump or related equipment, or water or water-borne material below the surface of the ground, including water which exerts pressure on or seeps or leaks through a building, sidewalk, driveway, foundation, swimming pool or other structure. Additionally, the policy excludes losses caused by war, communicable disease, or controlled substances. Finally, the policy will pay up to $2,500 for appliances, carpeting, and other household furnishings in each apartment on the "residence premises" regularly rented or held for rental to others, and up to 10% of the limit of liability that applies to Coverage A can be used for increased costs due to the enforcement of any ordinance or law which requires or regulates the construction, demolition, remodeling, renovation, or repair of a covered building or other structure, provided the dwelling is used only as a residence, or in part for use only as a residence, or in part, as an office, school, studio or private garage.

In [35]:
from llama_index import Prompt

my_qa_template = Prompt(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question below. Include a list of sources, i.e. TITLE, sub-title, A., 1., a, (1)., in your final answer. \n"
    "\n{query_str}\n"
)


In [36]:
CUSTOM_REFINE_PROMPT_TMPL = Prompt(
    "The original question is as follows: {query_str}\n"
    "We have provided an existing answer: {existing_answer}\n"
    "We have the opportunity to refine the existing answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better answer the question. "
    "If the context isn't useful, return the original answer. "
    "Include a list of sources, i.e. TITLE, sub-title, A., 1., a, (1)., in your final answer. "
    
)

In [55]:
from IPython.display import Markdown
from llama_index import (ServiceContext, 
                         StorageContext,
                         LLMPredictor,
                         load_index_from_storage)
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.prompts import DEFAULT_SUB_QUESTION_PROMPT_TMPL
from llama_index.query_engine import SubQuestionQueryEngine, CitationQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata
from langchain.chat_models import ChatOpenAI
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.response_synthesizers import get_response_synthesizer




llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

service_context = ServiceContext.from_defaults(
    llm=llm,
    callback_manager=callback_manager)


index = load_index_from_storage(
    StorageContext.from_defaults(persist_dir="./_policy_index_storage"),
    )

query_engine = index.as_query_engine(
    refine_template=CUSTOM_REFINE_PROMPT_TMPL,
    text_qa_template=my_qa_template)

# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="homeowners_policy", description="Homeowner's insurance policy document"
        ),
    )
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)

DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./_policy_index_storage\docstore.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./_policy_index_storage\docstore.json.
DEBUG:fsspec.local:open file: c:/Users/pdoub/Desktop/python_projects/junk-drawer/_policy_index_storage/docstore.json
open file: c:/Users/pdoub/Desktop/python_projects/junk-drawer/_policy_index_storage/docstore.json
DEBUG:llama_index.storage.kvstore.simple_kvstore:Loading llama_index.storage.kvstore.simple_kvstore from ./_policy_index_storage\index_store.json.
Loading llama_index.storage.kvstore.simple_kvstore from ./_policy_index_storage\index_store.json.
DEBUG:fsspec.local:open file: c:/Users/pdoub/Desktop/python_projects/junk-drawer/_policy_index_storage/index_store.json
open file: c:/Users/pdoub/Desktop/python_projects/junk-drawer/_policy_index_storage/index_store.json
DEBUG:llama_index.vector_stores.simple:Loading llama_index.vector_stores.simple 

In [56]:
response = await query_engine.aquery(
    "What are the considerations for determining if the policy does or does not cover damaged caused by collapse?"
)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": "Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\",\\n        \\"tool_name\\": \\"uber_10k\\"\\n    },\\n    {\\n        \\"sub_question\\": \\"What is the EBITDA of Uber\\",\\n        \\"tool_name\\": \\"uber_10k\\"\\n    },\\n    {\\n

In [57]:
Markdown(f"{response.response}")


The considerations for determining if the policy does or does not cover damage caused by collapse include: 
(1) The type of peril that caused the collapse; 
(2) Whether the presence of decay or insect/vermin damage was known to an "insured" prior to the collapse; 
(3) The weight of any contents, equipment, animals, or people that may have contributed to the collapse; 
(4) The weight of any rain, snow, ice, or sleet that may have collected on the roof; 
(5) The use of defective materials or methods in construction, remodeling, or renovation; 
(6) The hidden decay of a load-bearing member; 
(7) The use of defective design in construction, remodeling, or renovation; 
(8) The faulty workmanship in construction, remodeling, or renovation; 
(9) Whether the collapse was caused by an earthquake; and 
(10) Whether the collapse was caused by land shock waves or tremors before, during, or after a volcanic eruption.

In [58]:
Markdown(f"{response.source_nodes[0].node.get_text()}")

Sub question: What are the conditions for coverage of damage caused by collapse in the homeowner's policy?
Response: 
The conditions for coverage of damage caused by collapse in the homeowner's policy are: 
(1) The Perils Insured Against named under Coverage C; 
(2) Decay that is hidden from view, unless the presence of such decay is known to an "insured" prior to collapse; 
(3) Insect or vermin damage that is hidden from view, unless the presence of such damage is known to an "insured" prior to collapse; 
(4) Weight of contents, equipment, animals or people; 
(5) Weight of rain, snow, ice or sleet which collects on a roof; 
(6) Use of defective materials or methods in construction, remodeling or renovation if the collapse occurs during the course of the construction, remodeling or renovation; 
(7) Hidden decay of a load-bearing member; 
(8) Use of defective design in construction, remodeling or renovation if the collapse occurs during the course of the construction, remodeling or renovation; 
(9) Faulty workmanship in construction, remodeling or renovation if the collapse occurs during the course of the construction, remodeling or renovation; 
(10) Earthquake; or 
(11) Land shock waves or tremors before, during or after a volcanic eruption. 

Source: Sample_HO3_Policy, pages 7-7, Section 8. Collapse, a. and b.

In [59]:
pp.pprint_response(response, show_source=True)

Final Response: The considerations for determining if the policy does
or does not cover damage caused by collapse include:  (1) The type of
peril that caused the collapse;  (2) Whether the presence of decay or
insect/vermin damage was known to an "insured" prior to the collapse;
(3) The weight of any contents, equipment, animals, or people that may
have contributed to the collapse;  (4) The weight of any rain, snow,
ice, or sleet that may have collected on the roof;  (5) The use of
defective materials or methods in construction, remodeling, or
renovation;  (6) The hidden decay of a load-bearing member;  (7) The
use of defective design in construction, remodeling, or renovation;
(8) The faulty workmanship in construction, remodeling, or renovation;
(9) Whether the collapse was caused by an earthquake; and  (10)
Whether the collapse was caused by land shock waves or tremors before,
during, or after a volcanic eruption.
_____________________________________________________________________

In [62]:
print(response.source_nodes[0].node.get_text())

Sub question: What are the conditions for coverage of damage caused by collapse in the homeowner's policy?
Response: 
The conditions for coverage of damage caused by collapse in the homeowner's policy are: 
(1) The Perils Insured Against named under Coverage C; 
(2) Decay that is hidden from view, unless the presence of such decay is known to an "insured" prior to collapse; 
(3) Insect or vermin damage that is hidden from view, unless the presence of such damage is known to an "insured" prior to collapse; 
(4) Weight of contents, equipment, animals or people; 
(5) Weight of rain, snow, ice or sleet which collects on a roof; 
(6) Use of defective materials or methods in construction, remodeling or renovation if the collapse occurs during the course of the construction, remodeling or renovation; 
(7) Hidden decay of a load-bearing member; 
(8) Use of defective design in construction, remodeling or renovation if the collapse occurs during the course of the construction, remodeling or reno

In [61]:
print(response.source_nodes[1].node.get_text())

IndexError: list index out of range

In [63]:
query = "What are the duties owed of the policy holder and for the insurance company?"

response = response = await query_engine.aquery(query)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
DEBUG:openai:api_version=None data='{"prompt": "Given a user question, and a list of tools, output a list of relevant sub-questions that when composed can help answer the full user question:\\n\\n# Example 1\\n<Tools>\\n```json\\n{\\n    \\"uber_10k\\": \\"Provides information about Uber financials for year 2021\\",\\n    \\"lyft_10k\\": \\"Provides information about Lyft financials for year 2021\\"\\n}\\n```\\n\\n<User Question>\\nCompare and contrast the revenue growth and EBITDA of Uber and Lyft for year 2021\\n\\n\\n<Output>\\n```json\\n[\\n    {\\n        \\"sub_question\\": \\"What is the revenue growth of Uber\\",\\n        \\"tool_name\\": \\"uber_10k\\"\\n    },\\n    {\\n        \\"sub_question\\": \\"What is the EBITDA of Uber\\",\\n        \\"tool_name\\": \\"uber_10k\\"\\n    },\\n    {\\n

In [64]:
Markdown(f"{response.response}")


The duties owed of the policy holder are:
1. Give written notice to us or our agent as soon as is practical, which sets forth: a. The identity of the policy and the "named insured" shown in the Declarations; b. Reasonably available information on the time, place and circumstances of the "occurrence"; and c. Names and addresses of any claimants and witnesses;
2. Cooperate with us in the investigation, settlement or defense of any claim or suit;
3. Promptly forward to us every notice, demand, summons or other process relating to the "occurrence";
4. At our request, help us: a. To make settlement; b. To enforce any right of contribution or indemnity against any person or organization who may be liable to an "insured"; c. With the conduct of suits and attend hearings and trials; and d. To secure and give evidence and obtain the attendance of witnesses;
5. Give prompt notice to us or our agent;
6. Notify the police in case of loss by theft;
7. Notify the credit card or electronic fund transfer card or access device company in case of loss as provided for in E.6. Credit Card, Electronic Fund Transfer Card Or Access Device, Forgery And Counterfeit Money under Section I – Property Coverages;
8. Protect the property from further damage.

The duties owed by the insurance company are:
A. Insurable Interest And Limit Of Liability:
1. Not to be liable in any one loss to an "insured" for more than the amount of such "insured's" interest at the time of loss; or
2. Not to be liable for more than the applicable limit of liability.

B. Duties After Loss:
1. Give prompt notice to us or our agent;
2. Notify the police in case of loss by theft;
3. Notify the credit card or electronic fund transfer card or access device company in case of loss as provided for in E.6. Credit Card, Electronic Fund Transfer Card Or Access Device, Forgery And Counterfeit Money under Section I – Property Coverages;
4. Protect the property from further damage;
5. Provide us with a proof of loss;
6. Cooperate with us in the investigation, settlement or defense of any claim or suit;
7. Promptly forward to us every notice, demand, summons or other process relating to the "occ

In [66]:
Markdown(f"{response.source_nodes[0].node.get_text()}")

Sub question: What are the duties owed of the policy holder?
Response: 
1. Give written notice to us or our agent as soon as is practical, which sets forth: a. The identity of the policy and the "named insured" shown in the Declarations; b. Reasonably available information on the time, place and circumstances of the "occurrence"; and c. Names and addresses of any claimants and witnesses;
2. Cooperate with us in the investigation, settlement or defense of any claim or suit;
3. Promptly forward to us every notice, demand, summons or other process relating to the "occurrence";
4. At our request, help us: a. To make settlement; b. To enforce any right of contribution or indemnity against any person or organization who may be liable to an "insured"; c. With the conduct of suits and attend hearings and trials; and d. To secure and give evidence and obtain the attendance of witnesses;
5. Give prompt notice to us or our agent;
6. Notify the police in case of loss by theft;
7. Notify the credit card or electronic fund transfer card or access device company in case of loss as provided for in E.6. Credit Card, Electronic Fund Transfer Card Or Access Device, Forgery And Counterfeit Money under Section I – Property Coverages;
8. Protect the property from further damage.

In [67]:
Markdown(f"{response.source_nodes[1].node.text}")

Sub question: What are the duties owed of the insurance company?
Response: 
The duties owed by the insurance company are:

A. Insurable Interest And Limit Of Liability:
1. Not to be liable in any one loss to an "insured" for more than the amount of such "insured's" interest at the time of loss; or
2. Not to be liable for more than the applicable limit of liability.

B. Duties After Loss:
1. Give prompt notice to us or our agent;
2. Notify the police in case of loss by theft;
3. Notify the credit card or electronic fund transfer card or access device company in case of loss as provided for in E.6. Credit Card, Electronic Fund Transfer Card Or Access Device, Forgery And Counterfeit Money under Section I – Property Coverages;
4. Protect the property from further damage;
5. Provide us with a proof of loss;
6. Cooperate with us in the investigation, settlement or defense of any claim or suit;
7. Promptly forward to us every notice, demand, summons or other process relating to the "occurrence";
8. At our request, help us:
a. To make settlement;
b. To enforce any right of contribution or indemnity against any person or organization who may be liable to an "insured";
c. With the conduct of suits and attend hearings and trials; and
d. To secure and give evidence and obtain the attendance of witnesses.